# hvPlot.barbs

```{eval-rst}
.. currentmodule:: hvplot

.. automethod:: hvPlot.barbs
```

:::{note}
Wind barb plots require `geoviews` to be installed.
:::

## Backend-specific styling options

```{eval-rst}
.. backend-styling-options:: barbs
```

## Examples

### Basic wind barbs plot

In this example we create a simple DataFrame with 4 columns `x`, `y`, `angle` and `speed`. `x` and `y` represent the location of the wind barbs. `angle` defines the wind direction expressed in radians (meteorological convention: direction FROM which the wind is blowing, with 0 being North). `speed` defines the wind speed magnitude.

In [ ]:
import hvplot.pandas
import pandas as pd
import numpy as np

df = pd.DataFrame(dict(
    x=[-1, 0, 1, 0],
    y=[0, -1, 0, 1],
    angle=[0, np.pi/2, np.pi, 3*np.pi/2],
    speed=[5, 10, 15, 20],
))

df.hvplot.barbs(
    x="x", y="y", angle="angle", mag="speed",
    data_aspect=1, padding=0.4, width=400, height=400,
)

### Xarray example

In this example we also set the [`color`](option-color) to the speed variable and add a colorbar.

In [ ]:
import hvplot.xarray  # noqa
import numpy as np
import xarray as xr

def sample_wind_data(shape=(20, 30)):
    x = np.linspace(311.9, 391.1, shape[1])
    y = np.linspace(-23.6, 24.8, shape[0])
    x2d, y2d = np.meshgrid(x, y)
    u = 10 * (2 * np.cos(2 * np.deg2rad(x2d) + 3 * np.deg2rad(y2d + 30)) ** 2)
    v = 20 * np.cos(6 * np.deg2rad(x2d))
    return x, y, u, v

xs, ys, U, V = sample_wind_data()
mag = np.sqrt(U**2 + V**2)
angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)
ds = xr.Dataset({
    'speed': xr.DataArray(mag, dims=('y', 'x'), coords={'y': ys, 'x': xs}),
    'angle': xr.DataArray(angle, dims=('y', 'x'), coords={'y': ys, 'x': xs})
})

ds.hvplot.barbs(
    x='x', y='y', angle='angle', mag='speed',
    color='speed', cmap='viridis', colorbar=True,
    width=700, height=400
).opts(scale=0.3)

### Geographic example with Xarray

The `xarray.Dataset` constructed in this example has a `'crs'` key in its `attrs` dictionary, which lets us simply set `geo=True` to turn this plot into a correctly projected geographic plot overlaid on web map tiles.

In [ ]:
import cartopy.crs as ccrs
import hvplot.xarray  # noqa
import numpy as np
import xarray as xr

def sample_data(shape=(20, 30)):
    """
    Return ``(x, y, u, v, crs)`` of some vector data computed mathematically.
    The returned crs will be a rotated pole CRS, meaning that the vectors
    will be unevenly spaced in regular PlateCarree space.
    """
    crs = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)

    x = np.linspace(311.9, 391.1, shape[1])
    y = np.linspace(-23.6, 24.8, shape[0])
    x2d, y2d = np.meshgrid(x, y)
    u = 10 * (2 * np.cos(2 * np.deg2rad(x2d) + 3 * np.deg2rad(y2d + 30)) ** 2)
    v = 20 * np.cos(6 * np.deg2rad(x2d))
    return x, y, u, v, crs

xs, ys, U, V, crs = sample_data()
mag = np.sqrt(U**2 + V**2)
angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)
ds = xr.Dataset(
    {
        'speed': xr.DataArray(mag, dims=('y', 'x'), coords={'y': ys, 'x': xs}),
        'angle': xr.DataArray(angle, dims=('y', 'x'), coords={'y': ys, 'x': xs})
    },
    attrs={'crs': crs},
)

ds.hvplot.barbs(
    x="x", y="y", angle="angle", mag="speed",
    geo=True, tiles="CartoLight", width=700, height=400
).opts("WindBarbs", scale=0.3)